
- 복잡한 구조의 네트웍은 오히려 간단한 문제들에 있어서는 정확도가 더 떨어지는 모습
- 그냥 BN 직접 구현해보고 차이 비교해보기 위해서 복잡하게 만들어본 시도 


In [1]:

from mnist import MnistData
import tensorflow as tf
import numpy as np

pickle_file = 'notMNIST_sanit.pickle'
mnist = MnistData(pickle_file,one_hot=True)


Building Graph   

In [5]:

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

def conv_2_block(input,i_ch,o_ch):
    w_conv1 = weight_variable([3,3,i_ch,o_ch])
    b_conv1 = bias_variable([o_ch])
    h_conv1 = tf.nn.relu(conv2d(input,w_conv1)+b_conv1)
    w_conv2 = weight_variable([3,3,o_ch,o_ch])
    b_conv2 = bias_variable([o_ch])
    h_conv2 = tf.nn.relu(conv2d(h_conv1,w_conv2)+b_conv2)
    h_pool = max_pool_2x2(h_conv2)
    return h_pool

def conv_3_block(input,i_ch,o_ch):
    w_conv1 = weight_variable([3,3,i_ch,o_ch])
    b_conv1 = bias_variable([o_ch])
    h_conv1 = tf.nn.relu(conv2d(input,w_conv1)+b_conv1)
    w_conv2 = weight_variable([3,3,o_ch,o_ch])
    b_conv2 = bias_variable([o_ch])
    h_conv2 = tf.nn.relu(conv2d(h_conv1,w_conv2)+b_conv2)
    w_conv3 = weight_variable([3,3,o_ch,o_ch])
    b_conv3 = bias_variable([o_ch])
    h_conv3 = tf.nn.relu(conv2d(h_conv2,w_conv3)+b_conv3)
    h_pool = max_pool_2x2(h_conv3)
    return h_pool

x = tf.placeholder(tf.float32,[None,28,28,1])
y_ = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

with tf.name_scope('conv_nets'):
    conv_block1 = conv_2_block(x,1,32)
    conv_block2 = conv_2_block(conv_block1,32,64)
    conv_block3 = conv_3_block(conv_block2,64,128)
    conv_block4 = conv_3_block(conv_block3,128,256)

print(conv_block4)
print(conv_block4.get_shape())
print(conv_block4.get_shape()[-1])
print(conv_block4.get_shape().as_list()[1:])

with tf.name_scope('fc-layers'):
    w_fc1 = weight_variable([2*2*256,512])
    b_fc1 = weight_variable([512])
    h_conv_flatten = tf.reshape(conv_block4,[-1,2*2*256])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv_flatten,w_fc1)+b_fc1)
    w_fc2 = weight_variable([512,512])
    b_fc2 = bias_variable([512])
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1,w_fc2)+b_fc2)
    w_fc3 = weight_variable([512,10])
    b_fc3 = bias_variable([10])
    y = tf.matmul(h_fc2,w_fc3)+b_fc3


Tensor("conv_nets_3/MaxPool_3:0", shape=(?, 2, 2, 256), dtype=float32)
(?, 2, 2, 256)
256
[2, 2, 256]


휴... 듸게 바보같은 노가다 코딩이긴 한데 일단 좀 복잡하게 conv_net 10개 깔고 fully connected 2개 달았음 ㅠ.ㅠ

In [ ]:
batch_size = 100
total_batch = int(mnist.train_data.data_length/batch_size)
epoch = 10

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=y_))
    tf.summary.scalar('cross_entropy',cross_entropy)

with tf.name_scope('train'):
    global_step = tf.Variable(0,trainable=False)
    starter_learning_rate = 1e-4
    learning_rate = tf.train.exponential_decay(starter_learning_rate,global_step,5000,0.96,staircase=True)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,global_step=global_step)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy',accuracy)
    tf.summary.scalar('learning_rate',learning_rate)

sess = tf.InteractiveSession()

# merge all the summaries

merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter("./summary/train",sess.graph)
test_writer = tf.summary.FileWriter("./summary/test",sess.graph)

sess.run(tf.global_variables_initializer())


자 여기서 부터는 일단 train 을 한번 돌려보자... 

In [ ]:

for i_epoch in range(epoch):
    for i in range(total_batch):
        batch = mnist.train_data.next_batch(batch_size)
        xs = np.reshape(batch[0],(-1,28,28,1))
        ys = batch[1]
        summary, _ = sess.run([merged,train_step],feed_dict = { x: xs, y_: ys, keep_prob: 0.75})
        train_writer.add_summary(summary,i_epoch*total_batch+i)

        if i % 1000 == 0:
            train_accuracy = accuracy.eval(feed_dict = { x: xs, y_: ys, keep_prob: 1.0})
            print("epoch %d, batch_iteration %d, train_accuracy %g" % (i_epoch,i,train_accuracy))
            print("cross entropy %g" % sess.run(cross_entropy, feed_dict = { x:xs,y_:ys, keep_prob: 1.0}))
            valid_accuracy = accuracy.eval(feed_dict = { x: np.reshape(mnist.valid_data.images,(-1,28,28,1)), y_: mnist.valid_data.labels, keep_prob: 1.0})
            print("validation accuracy %g" % valid_accuracy)

print("test accuracy %g" %  accuracy.eval(feed_dict = { x: np.reshape(mnist.test_data.images,(-1,28,28,1)), y_: mnist.test_data.labels, keep_prob: 1.0}))




- dropout, learning rate decay 이런것들이 오히려 나쁜 영향을 주는 듯... 가장 간단한 기본구조였을때의 test accuracy 는 96~97%사이
- AdamOptimizer 랑 그냥 GradientDescentOptimizer 랑도 차이를 꽤 만들어내는 듯(?) -- crossentropy 를 계산하는 값도 다르게 진행됨 
- 실험환경들 기록해두도록 할 것..  
